In [ ]:
# LOADING HMDB51
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! apt-get install libavformat-dev libavdevice-dev
! pip install av==6.2.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libavformat-dev is already the newest version (7:3.4.8-0ubuntu0.2).
libavformat-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libavfilter-dev libpostproc-dev
The following NEW packages will be installed:
  libavdevice-dev libavfilter-dev libpostproc-dev
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 1,154 kB of archives.
After this operation, 5,444 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libpostproc-dev amd64 7:3.4.8-0ubuntu0.2 [51.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libavfilter-dev amd64 7:3.4.8-0ubuntu0.2 [1,016 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd

In [ ]:
! pip install -q kaggle
! cp '/content/drive/MyDrive/kaggle.json' '/content'
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d pevogam/ucf101
! unzip ucf101.zip
! rm ucf101.zip

Streaming output truncated to the last 5000 lines.
  inflating: UCF101/UCF-101/PlayingGuitar/v_PlayingGuitar_g01_c02.avi  
  inflating: UCF101/UCF-101/PlayingGuitar/v_PlayingGuitar_g01_c03.avi  
  inflating: UCF101/UCF-101/PlayingGuitar/v_PlayingGuitar_g01_c04.avi  
  inflating: UCF101/UCF-101/PlayingGuitar/v_PlayingGuitar_g01_c05.avi  
  inflating: UCF101/UCF-101/PlayingGuitar/v_PlayingGuitar_g01_c06.avi  
  inflating: UCF101/UCF-101/PlayingGuitar/v_PlayingGuitar_g02_c01.avi  
  inflating: UCF101/UCF-101/PlayingGuitar/v_PlayingGuitar_g02_c02.avi  
  inflating: UCF101/UCF-101/PlayingGuitar/v_PlayingGuitar_g02_c03.avi  
  inflating: UCF101/UCF-101/PlayingGuitar/v_PlayingGuitar_g02_c04.avi  
  inflating: UCF101/UCF-101/PlayingGuitar/v_PlayingGuitar_g03_c01.avi  
  inflating: UCF101/UCF-101/PlayingGuitar/v_PlayingGuitar_g03_c02.avi  
  inflating: UCF101/UCF-101/PlayingGuitar/v_PlayingGuitar_g03_c03.avi  
  inflating: UCF101/UCF-101/PlayingGuitar/v_PlayingGuitar_g03_c04.avi  
  inflating: 

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
from torch.autograd import Variable
from torch.utils.data import random_split, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision
from torchvision import get_video_backend
from torchvision.models.video import r3d_18 
from torchvision import transforms
import os
from tqdm.auto import tqdm
import numpy as np
import time
import av
import random
import shutil
 
from collections import Ord9eredDict
 
SEED = 491
torch.manual_seed(SEED)

In [ ]:

# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
print(len(GPUs))
gpu = GPUs[0]

  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=a2dce082f96c3e0b80f1d1cac967732ae0996a03426040f2599d432a7babbd79
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
1


In [ ]:
! wget https://raw.githubusercontent.com/pytorch/vision/6de158c473b83cf43344a0651d7c01128c7850e6/references/video_classification/transforms.py
import transforms as T

--2021-05-14 08:18:11--  https://raw.githubusercontent.com/pytorch/vision/6de158c473b83cf43344a0651d7c01128c7850e6/references/video_classification/transforms.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3102 (3.0K) [text/plain]
Saving to: ‘transforms.py’

transforms.py       100%[===================>]   3.03K  --.-KB/s    in 0s      

2021-05-14 08:18:12 (38.9 MB/s) - ‘transforms.py’ saved [3102/3102]



In [ ]:
def custom_collate(batch):
    filtered_batch = []
    for video, _, label in batch:
        filtered_batch.append((video, label))
    return torch.utils.data.dataloader.default_collate(filtered_batch)

In [ ]:

num_frames = 16 # 16
clip_steps = 50
num_workers = 2
pin_memory = True
train_tfms = torchvision.transforms.Compose([
                                 T.ToFloatTensorInZeroOne(),
                                 T.Resize((128, 128)),
                                 T.RandomHorizontalFlip(),
                                 T.Normalize(mean=[0, 0, 0], std=[1, 1, 1]),
                                 T.CenterCrop((112, 112))
                               ])  
test_tfms =  torchvision.transforms.Compose([
                                             T.ToFloatTensorInZeroOne(),
                                             T.Resize((128, 128)),
                                             T.Normalize(mean=[0, 0, 0], std=[1, 1, 1]),
                                             T.CenterCrop((112, 112))
                                             ])
hmdb51_train = torchvision.datasets.UCF101('UCF101/UCF-101/', 'UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/', num_frames,
                                                step_between_clips = clip_steps, fold=1, train=True,
                                                transform=train_tfms)


hmdb51_test = torchvision.datasets.UCF101('UCF101/UCF-101/', 'UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/', num_frames,
                                                step_between_clips = clip_steps, fold=1, train=False,
                                                transform=test_tfms)
      
total_train_samples = len(hmdb51_train)

print(f"number of train samples {total_train_samples}")
print(f"number of test samples {len(hmdb51_test)}")


number of train samples 37428
number of test samples 14621


In [ ]:
bs = 4
lr = 1e-2
gamma = 0.1
step_size = 5
total_epochs = 10
config = {}

kwargs = {'num_workers':num_workers, 'pin_memory':True} if torch.cuda.is_available() else {'num_workers':num_workers}

  
train_loader = DataLoader(hmdb51_train, batch_size=64, shuffle=True,pin_memory=True,collate_fn=custom_collate)
test_loader  = DataLoader(hmdb51_test, batch_size=64, shuffle=False,pin_memory=True,collate_fn=custom_collate)


In [ ]:
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


def conv_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv3d(inp, oup, kernel_size=3, stride=stride, padding=(1, 1, 1), bias=False),
        nn.BatchNorm3d(oup),
        nn.ReLU6(inplace=True)
    )


def conv_1x1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv3d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm3d(oup),
        nn.ReLU6(inplace=True)
    )


class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        self.stride = stride

        hidden_dim = round(inp * expand_ratio)
        self.use_res_connect = self.stride == (1, 1, 1) and inp == oup

        if expand_ratio == 1:
            self.conv = nn.Sequential(
                # dw
                nn.Conv3d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm3d(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear
                nn.Conv3d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm3d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # pw
                nn.Conv3d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm3d(hidden_dim),
                nn.ReLU6(inplace=True),
                # dw
                nn.Conv3d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm3d(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear
                nn.Conv3d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm3d(oup),
            )

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)


class MobileNetV2(nn.Module):
    def __init__(self, num_classes=1000, sample_size=224, width_mult=1.):
        super(MobileNetV2, self).__init__()
        block = InvertedResidual
        input_channel = 32
        last_channel = 1280
        inverted_residual_setting = [
            # t, c, n, s
            [1,  16, 1, (1,1,1)],
            [6,  24, 2, (2,2,2)],
            [6,  32, 3, (2,2,2)],
            [6,  64, 4, (2,2,2)],
            [6,  96, 3, (1,1,1)],
            [6, 160, 3, (2,2,2)],
            [6, 320, 1, (1,1,1)],
        ]

        # building first layer
        assert sample_size % 16 == 0.
        input_channel = int(input_channel * width_mult)
        self.last_channel = int(last_channel * width_mult) if width_mult > 1.0 else last_channel
        self.features = [conv_bn(3, input_channel, (1,2,2))]
        # building inverted residual blocks
        for t, c, n, s in inverted_residual_setting:
            output_channel = int(c * width_mult)
            for i in range(n):
                stride = s if i == 0 else (1,1,1)
                self.features.append(block(input_channel, output_channel, stride, expand_ratio=t))
                input_channel = output_channel
        # building last several layers
        self.features.append(conv_1x1x1_bn(input_channel, self.last_channel))
        # make it nn.Sequential
        self.features = nn.Sequential(*self.features)

        # building classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.8),
            nn.Linear(self.last_channel, num_classes),
        )

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = F.avg_pool3d(x, x.data.size()[-3:])
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.kernel_size[2] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                n = m.weight.size(1)
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()


def get_fine_tuning_parameters(model, ft_portion):
    if ft_portion == "complete":
        return model.parameters()

    elif ft_portion == "last_layer":
        ft_module_names = []
        ft_module_names.append('classifier')

        parameters = []
        for k, v in model.named_parameters():
            for ft_module in ft_module_names:
                if ft_module in k:
                    parameters.append({'params': v})
                    break
            else:
                parameters.append({'params': v, 'lr': 0.0})
        return parameters

    else:
        raise ValueError("Unsupported ft_portion: 'complete' or 'last_layer' expected")

    
def get_model(**kwargs):
    """
    Returns the model.
    """
    model = MobileNetV2(**kwargs)
    return model



In [ ]:
class AverageMeter(object):
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def update_acc(self, val, n=1):
        self.val = val/n
        self.sum += val
        self.count += n
        self.avg = self.sum / self.count

def calculate_accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


def save_checkpoint(state, is_best,path):
    torch.save(state, '%s' % (path))
    if is_best:
        shutil.copyfile('%s' % (path),'/content/drive/MyDrive/UCF_best.pth')



In [ ]:
def train_epoch(epoch, data_loader, model, optimizer):
    print('train at epoch {}'.format(epoch))

    model.train()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    end_time = time.time()
    for i, data in enumerate(data_loader):
        inputs, targets = data[0], data[-1]
        data_time.update(time.time() - end_time)

        if torch.cuda.is_available():
          targets = targets.cuda()
          inputs = inputs.cuda()
        inputs = Variable(inputs)
        targets = Variable(targets)
        outputs = model(inputs)
        criterion = nn.CrossEntropyLoss()
        loss = criterion(outputs, targets)

        losses.update(loss.data, inputs.size(0))
        prec1, prec5 = calculate_accuracy(outputs.data, targets.data, topk=(1,5))
        top1.update(prec1, inputs.size(0))
        top5.update(prec5, inputs.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end_time)
        end_time = time.time()

        if i % 100 ==0:
            print('Epoch: [{0}][{1}/{2}]\t lr: {lr:.5f}\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.5f} ({top1.avg:.5f})\t'
                  'Prec@5 {top5.val:.5f} ({top5.avg:.5f})'.format(
                  epoch,
                  i,
                  len(data_loader),
                  batch_time=batch_time,
                  data_time=data_time,
                  loss=losses,
                  top1=top1,
                  top5=top5,
                  lr=optimizer.param_groups[0]['lr']))

    

In [ ]:
def val_epoch(epoch, data_loader, model):
  print('validation at epoch {}'.format(epoch))

  model.eval()

  batch_time = AverageMeter()
  data_time = AverageMeter()
  losses = AverageMeter()
  top1 = AverageMeter()
  top5 = AverageMeter()

  end_time = time.time()
  for i, data in enumerate(data_loader):
      inputs, targets = data[0], data[-1]
      data_time.update(time.time() - end_time)

      if torch.cuda.is_available():
          targets = targets.cuda()
          inputs = inputs.cuda()
      with torch.no_grad():
          inputs = Variable(inputs)
          targets = Variable(targets)
      outputs = model(inputs)
      criterion = nn.CrossEntropyLoss()
      loss = criterion(outputs, targets)
      prec1, prec5 = calculate_accuracy(outputs.data, targets.data, topk=(1,5))
      top1.update(prec1, inputs.size(0))
      top5.update(prec5, inputs.size(0))

      losses.update(loss.data, inputs.size(0))

      batch_time.update(time.time() - end_time)
      end_time = time.time()

      print('Epoch: [{0}][{1}/{2}]\t'
            'Time {batch_time.val:.5f} ({batch_time.avg:.5f})\t'
            'Data {data_time.val:.5f} ({data_time.avg:.5f})\t'
            'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
            'Prec@1 {top1.val:.5f} ({top1.avg:.5f})\t'
            'Prec@5 {top5.val:.5f} ({top5.avg:.5f})'.format(
            epoch,
            i + 1,
            len(data_loader),
            batch_time=batch_time,
            data_time=data_time,
            loss=losses,
            top1=top1,
            top5=top5))
  return losses.avg.item(), top1.avg.item()

In [ ]:
def test(data_loader, model):
    print('test')

    model.eval()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    end_time = time.time()
    
    with torch.no_grad():
        for i, data in enumerate(data_loader):
            inputs, targets = data[0], data[-1]
            data_time.update(time.time() - end_time)
            if torch.cuda.is_available():
                targets = targets.cuda()
                inputs = inputs.cuda()
           
            inputs = Variable(inputs)
            targets = Variable(targets)
            outputs = model(inputs)
            criterion = nn.CrossEntropyLoss()
            loss = criterion(outputs, targets)
            prec1, prec5 = calculate_accuracy(outputs.data, targets.data, topk=(1,5))
            top1.update(prec1, inputs.size(0))
            top5.update(prec5, inputs.size(0))

            losses.update(loss.data, inputs.size(0))

            batch_time.update(time.time() - end_time)
            end_time = time.time()
            
            if(i + 1 == len(data_loader)):
                print('[{0}/{1}]\t'
                'Time {batch_time.val:.5f} ({batch_time.avg:.5f})\t'
                'Data {data_time.val:.5f} ({data_time.avg:.5f})\t'
                'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                'Prec@1 {top1.val:.5f} ({top1.avg:.5f})\t'
                'Prec@5 {top5.val:.5f} ({top5.avg:.5f})'.format(
                i + 1,
                len(data_loader),
                batch_time=batch_time,
                data_time=data_time,
                loss=losses,
                top1=top1,
                top5=top5))
    return top1.avg.item()       

In [ ]:
path = '/content/drive/MyDrive/UCF.pth'

model = get_model(num_classes=101, sample_size=112, width_mult=1.)
checkpoint = torch.load(path)
model.load_state_dict(checkpoint['state_dict'])
if torch.cuda.is_available():
  model = model.cuda()

optimizer = torch.optim.SGD(
            model.parameters(),
            lr=lr,
            momentum=0.9,
            dampening=0.9,
            weight_decay=1e-3)
scheduler = ReduceLROnPlateau(
            optimizer, 'min', patience=7)

optimizer.load_state_dict(checkpoint['optimizer'])
best_prec1 = checkpoint['best_prec1']

In [ ]:
for param in model.parameters():
    param.requires_grad = True

print(sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6)

2.483429


In [ ]:

print("Launching Action Recognition Model training")
for i in range(20):
  train_epoch(i, train_loader, model, optimizer)
  state = {
      'state_dict': model.state_dict(),
      'optimizer': optimizer.state_dict(),
      'best_prec1': best_prec1
      }
  save_checkpoint(state, False, path)
  prec1 = test(test_loader, model)
  is_best = prec1 > best_prec1
  best_prec1 = max(prec1, best_prec1)
  state = {
  'state_dict': model.state_dict(),
  'optimizer': optimizer.state_dict(),
  'best_prec1': best_prec1
  }
  save_checkpoint(state, is_best, path)

Launching Action Recognition Model training
train at epoch 0


/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:159: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "


Epoch: [0][0/585]	 lr: 0.01000	Time 5.800 (5.800)	Data 4.879 (4.879)	Loss 0.9763 (0.9763)	Prec@1 67.18750 (67.18750)	Prec@5 98.43750 (98.43750)
Epoch: [0][100/585]	 lr: 0.01000	Time 5.701 (5.739)	Data 4.969 (5.021)	Loss 0.5999 (1.0301)	Prec@1 82.81250 (72.13799)	Prec@5 100.00000 (93.45606)
Epoch: [0][200/585]	 lr: 0.01000	Time 5.491 (5.721)	Data 4.775 (5.005)	Loss 0.2752 (0.8454)	Prec@1 95.31250 (77.30099)	Prec@5 100.00000 (95.37469)
Epoch: [0][300/585]	 lr: 0.01000	Time 5.768 (5.692)	Data 5.039 (4.978)	Loss 0.5982 (0.7456)	Prec@1 84.37500 (80.25851)	Prec@5 98.43750 (96.22092)
Epoch: [0][400/585]	 lr: 0.01000	Time 5.641 (5.700)	Data 4.934 (4.986)	Loss 0.4297 (0.6767)	Prec@1 87.50000 (82.21634)	Prec@5 100.00000 (96.76590)
Epoch: [0][500/585]	 lr: 0.01000	Time 5.707 (5.668)	Data 5.016 (4.955)	Loss 0.2831 (0.6267)	Prec@1 92.18750 (83.60155)	Prec@5 100.00000 (97.14321)
test
[229/229]	Time 1.48061 (4.01919)	Data 1.30125 (3.56885)	Loss 2.5267 (0.8437)	Prec@1 13.79310 (76.71841)	Prec@5 58.620

In [ ]:
test(test_loader, model)